In [2]:
import pandas as pd

from sklearn.linear_model import LinearRegression

In [5]:
X_train_read = pd.read_csv('dataset/X_test_sat4_samp20000.csv', iterator=True, chunksize=1000)
X_train = pd.concat(X_train_read)

# y_train = pd.read_csv('dataset/y_train_sat4.csv')
print('Done.')

Done.


In [ ]:
print(X_train.info())
print(y_train.info())

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

print(r'bias:\t{}'.format(lr.intercept_))
print(r'weights:\t{}'.format(lr.coef_))